# Feature Engineering
Explanation:

## Import Packages

In [2]:
from importnb import Notebook
import pandas as pd
import numpy as np
from rapidfuzz import fuzz, process

## Dataframe import from 'DataPreprocessing'

In [3]:
with Notebook():
    from DataPreprocessing_Pipeline import companies, funding, investors, founders

## New Features: Companies <> Investment Rounds
Explanation:

### Feature 1: Months Until First Round
Number of months between the funding date and the first investment round. Explanation: The table 'all_rounds' has every investment round made during the existence of the companies. We already mapped the respectively funding round to the startup. 

In [4]:
def calculate_months_until_first_round(company_id, founded_date):
    rounds = funding[funding['Company ID'] == company_id]

    if rounds.empty:
        return np.nan

    first_round_date = rounds['Announced Date'].min()

    delta_months = (first_round_date.year - founded_date.year) * 12 + (first_round_date.month - founded_date.month)
    return max(delta_months, 0)

companies['Months until First Round'] = companies.apply(
    lambda row: calculate_months_until_first_round(row['ID'], row['Founded Date']),
    axis=1
)

companies['Months until First Round'] = companies['Months until First Round'].fillna(-1)

### Feature 2: Grant Y/N
If the company received a Grant, they get a Yes (1). If not, a No (0). For VCs it is desirable to have a company with a Grant because it leverages the invested money as a Grant do not dilute existing shareholders.

In [5]:
def check_grant_for_company(company_id):
    company_rounds = funding[funding['Company ID'] == company_id]
    has_grant = any(company_rounds['Funding Type'] == 'Grant')
    return 1 if has_grant else 0

companies['Grant Y/N'] = companies['ID'].apply(check_grant_for_company)

### Feature 3: Last Round Type
Explanation:

In [6]:
funding['Announced Date'] = pd.to_datetime(funding['Announced Date'])

last_round = (
    funding.sort_values(by=['Company ID', 'Announced Date'])
    .groupby('Company ID')
    .last()
    .reset_index()
)

last_round = last_round[['Company ID', 'Funding Type']]

companies = companies.merge(
    last_round,
    left_on='ID',
    right_on='Company ID',
    how='left'
)

companies = companies.rename(columns={'Funding Type': 'Last Round Type'})

companies.drop(columns=['Company ID'], inplace=True)

companies['Last Round Type'] = companies['Last Round Type'].fillna('No Funding')

companies.drop(columns=['Last Funding Type'], inplace=True)

### Feature 4: Acquisition Status 'Was Acquired', 'Made Acquisitions'
Explanation:

In [7]:
if 'Acquisition Status' in companies.columns:
    unique_values = companies['Acquisition Status'].dropna().unique()
    print(unique_values)

['Was Acquired' 'Made Acquisitions' 'Made Acquisitions, Was Acquired']


In [8]:
companies['Was Acquired'] = 0
companies['Made Acquisitions'] = 0

companies.loc[companies['Acquisition Status'] == 'Was Acquired', 'Was Acquired'] = 1
companies.loc[companies['Acquisition Status'] == 'Made Acquisitions', 'Made Acquisitions'] = 1
companies.loc[companies['Acquisition Status'] == 'Made Acquisitions, Was Acquired', ['Made Acquisitions', 'Was Acquired']] = 1

companies.drop(columns=['Acquisition Status'], inplace=True)

### Feature 5: Funding phases
Explanation:

In [9]:
if 'Last Round Type' in companies.columns:
    distinct_funding_types = companies['Last Round Type'].dropna().unique()
    print(distinct_funding_types)
else:
    print("The column 'Funding Type' does not exist in the DataFrame.")

['Seed' 'No Funding' 'Venture - Series Unknown' 'Series A' 'Series B'
 'Convertible Note' 'Angel' 'Post-IPO Debt' 'Private Equity' 'Pre-Seed'
 'Grant' 'Series C' 'Series F' 'Series D' 'Post-IPO Secondary' 'Series E'
 'Post-IPO Equity']


In [10]:
funding_columns = ['Project Funding', 'Startup Funding', 'Growth Funding', 'Expansion Funding', 'Exit Funding']
for col in funding_columns:
    companies[col] = 0

funding_mapping = {
    'Project Funding': ['Angel', 'Pre-Seed', 'Convertible Note', 'Grant', 'Venture - Series Unknown'],
    'Startup Funding': ['Seed', 'Venture - Series Unknown'],
    'Growth Funding': ['Series A', 'Series B', 'Series C'],
    'Expansion Funding': ['Series D', 'Series E', 'Series F'],
    'Exit Funding': ['Private Equity', 'Post-IPO Secondary', 'Post-IPO Equity']
}

for funding_type, types in funding_mapping.items():
    companies.loc[companies['Last Round Type'].isin(types), funding_type] = 1

companies.loc[companies['Exit Funding'] == 1, funding_columns[:-1]] = 1
companies.loc[companies['Expansion Funding'] == 1, ['Growth Funding', 'Startup Funding', 'Project Funding']] = 1
companies.loc[companies['Growth Funding'] == 1, ['Startup Funding', 'Project Funding']] = 1
companies.loc[companies['Startup Funding'] == 1, 'Project Funding'] = 1

### Feature 6: HighFunding (Series B or higher)

In [11]:
high_funding_rounds = ['Series B', 'Series C', 'Series D', 'Series E', 'Series F', 'Private Equity', 'Post-IPO Equity']
companies['HighFunding'] = companies['Last Round Type'].isin(high_funding_rounds).astype(int)

In [277]:
companies

,ID,Organization Name,Industries,Description,CB Rank (Company),Founded Date,Exit Date,Website,Twitter,Facebook,...,Grant Y/N,Last Round Type,Was Acquired,Made Acquisitions,Project Funding,Startup Funding,Growth Funding,Expansion Funding,Exit Funding,HighFunding
0,000001,2trde,"[Automotive, Software]",2trde develops a software solution designed fo...,"57,123",2017-01-01,NaT,https://www.2trde.com,NaN,NaN,...,0,Seed,0,0,1,1,0,0,0,0
1,000002,36ZERO Vision,"[Artificial Intelligence (AI), Computer Vision...",36ZEROVision is an AI-powered visual inspectio...,"51,326",2019-01-01,NaT,https://36zerovision.com/,NaN,NaN,...,0,Seed,0,0,1,1,0,0,0,0
2,000003,3Bears Foods,[Food and Beverage],3Bears Foods enable a balanced and delicious b...,"275,817",2015-01-01,NaT,https://3bears.de/,NaN,https://www.facebook.com/3bearsporridge/,...,0,Seed,0,0,1,1,0,0,0,0
3,000004,3dTrust,"[3D Printing, Manufacturing, Software]",3dTrust helps companies integrate 3D printing ...,"134,694",2015-01-01,NaT,http://3dtrust.de,https://twitter.com/3dTrust,NaN,...,0,No Funding,0,0,0,0,0,0,0,0
4,000005,abaut,"[Analytics, Artificial Intelligence (AI), Cons...",abaut builds a SaaS that enables businesses al...,"219,525",2017-07-21,NaT,https://abaut.de,NaN,NaN,...,0,Seed,0,0,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1508,001506,SAYM,"[Apps, B2B, B2C, Human Resources, Mobile Apps,...",The SAYM platform for swarm mobility defines t...,"166,910",2019-01-01,NaT,https://www.saym.io/,https://twitter.com/saym_io,https://www.facebook.com/saymcommute/,...,1,Grant,0,0,1,0,0,0,0,0
1509,001507,SONAH GmbH,"[Apps, Artificial Intelligence (AI), Computer ...",SONAH developed a flexible embedded vision sen...,"121,752",2016-01-01,NaT,http://www.sonah.tech,NaN,NaN,...,0,Seed,0,0,1,1,0,0,0,0
1510,001508,Taxy.io,"[FinTech, Legal Tech, Machine Learning, Software]",Taxy.io builds the leading platform for B2B ta...,"220,816",2019-01-01,NaT,https://www.taxy.io/,https://twitter.com/taxy_io,NaN,...,0,Seed,0,0,1,1,0,0,0,0
1511,001509,TRINKKOST GmbH,"[Agriculture, Consumer Goods, Fitness, Food an...",TRINKKOST is a food supplement manufacturing c...,"907,817",2016-01-01,NaT,http://www.trinkkost.de,https://www.twitter.com/trinkkost,https://www.facebook.com/trinkkost/,...,0,Venture - Series Unknown,0,0,1,1,0,0,0,0


In [278]:
funding

,Funding ID,Organization Name,Funding Type,Money Raised,Announced Date,Lead Investors,Number of Investors,Investor Names,Company ID
0,000001,4stop,Series A,2500000.0,2019-05-15,Ventech,1.0,[Ventech],000000
1,000002,aiconix GmbH,Convertible Note,NaN,2019-06-17,NaN,NaN,[],001193
2,000003,aiconix GmbH,Pre-Seed,300000.0,2019-08-28,NaN,NaN,[],001193
3,000004,AutLay,Seed,NaN,2019-05-22,Crew Ventures,1.0,[Crew Ventures],001196
4,000005,Buynomics,Pre-Seed,NaN,2019-09-01,DvH Ventures,2.0,"[DvH Ventures, Tomahawk.VC]",001205
...,...,...,...,...,...,...,...,...,...
6863,006864,VAMOS.ai,Seed,NaN,2020-01-01,DDG AG,1.0,[DDG AG],000000
6864,006865,WeProfit,Pre-Seed,272000.0,2021-05-28,NaN,5.0,"[Ara Abrahamyan, Armen Kocharyan, Clemens Boll...",000298
6865,006866,WindStar Medical GmbH,Seed,NaN,2020-12-07,Project A Ventures,1.0,[Project A Ventures],000000
6866,006867,WorkMentality Foundation,Seed,NaN,2023-03-01,NaN,1.0,[Deutsche Bank],000000


### Feature 7: Average Time To Next Round
Explanation:

In [12]:
funding['Announced Date'] = pd.to_datetime(funding['Announced Date'])
companies['Founded Date'] = pd.to_datetime(companies['Founded Date'])

def calculate_avg_time_to_next_round(company_id, company_founded_date, funding_df):
    company_funding = funding_df[funding_df['Company ID'] == company_id]

    if len(company_funding) == 1:
        time_diff = (company_funding['Announced Date'].iloc[0] - company_founded_date).days
        avg_time_to_next_round = time_diff / 30
    elif len(company_funding) >= 2:
        time_diffs = []
        first_round_diff = (company_funding['Announced Date'].iloc[0] - company_founded_date).days
        time_diffs.append(first_round_diff)
        company_funding = company_funding.sort_values('Announced Date')
        for i in range(1, len(company_funding)):
            time_diff = (company_funding['Announced Date'].iloc[i] - company_funding['Announced Date'].iloc[i-1]).days
            time_diffs.append(time_diff)
        avg_time_to_next_round = sum(time_diffs) / len(time_diffs) / 30
    else:
        avg_time_to_next_round = None

    if pd.isna(avg_time_to_next_round) or avg_time_to_next_round < -1:
        avg_time_to_next_round = -1

    return avg_time_to_next_round

companies['Average Time To Next Round'] = companies.apply(
    lambda row: calculate_avg_time_to_next_round(row['ID'], row['Founded Date'], funding), axis=1
)

### Feature 8: Average Funding Size
Explanation:

In [13]:
funding_summary = funding.groupby('Company ID').agg(
    TotalMoneyRaised=('Money Raised', 'sum'),
    TotalRounds=('Money Raised', 'count')
).reset_index()

funding_summary['Average Funding Size'] = funding_summary['TotalMoneyRaised'] / funding_summary['TotalRounds']

companies = companies.merge(
    funding_summary[['Company ID', 'Average Funding Size']],
    left_on='ID',
    right_on='Company ID',
    how='left'
)

companies['Average Funding Size'] = companies['Average Funding Size'].fillna(0)

companies.drop(columns=['Company ID'], inplace=True)

## New Features: Companies <> Investors
Explanation:

### Feature 1-4: Average Number of Investments by Investors
Explanation:

In [14]:
funding_exploded = funding.explode('Investor Names')
funding_exploded['Investor Names'] = funding_exploded['Investor Names'].str.strip()
investors['Organization/Person Name'] = investors['Organization/Person Name'].str.strip()

merged = funding_exploded.merge(
    investors,
    left_on='Investor Names',
    right_on='Organization/Person Name',
    how='left'
)

columns_to_process = ['Number of Investments', 'Number of Exits', 'Number of Lead Investments', 'Number of Portfolio Organizations']
for col in columns_to_process:
    if col in merged.columns:
        merged[col] = pd.to_numeric(merged[col], errors='coerce')

not_found_investors = merged[merged['Number of Investments'].isnull()]['Investor Names'].unique()
if len(not_found_investors) > 0:
    print(f"{len(not_found_investors)} investors could not be found.")
else:
    print("All investors were successfully found.")

not_found_investors_df = pd.DataFrame(not_found_investors, columns=['Investor Names'])

if 'Company ID' in companies.columns:
    companies = companies.drop(columns=['Company ID'])

for col in columns_to_process:
    col_average = merged.groupby('Company ID')[col].mean().reset_index()
    companies = companies.merge(
        col_average,
        left_on='ID',
        right_on='Company ID',
        how='left',
        suffixes=('', '_drop')
    )
    companies[col] = companies[col].fillna(0)
    companies = companies.rename(columns={col: f'Average {col} by Investors'})

companies = companies.drop(columns=[col for col in companies.columns if col.endswith('_drop')], errors='ignore')


3608 investors could not be found.


### Feature 5: Origin Country of Investors
Explanation:

In [15]:
funding_exploded = funding.explode('Investor Names')
funding_exploded['Investor Names'] = funding_exploded['Investor Names'].str.strip()
investors['Organization/Person Name'] = investors['Organization/Person Name'].str.strip()

merged = funding_exploded.merge(
    investors[['Organization/Person Name', 'Country']],
    left_on='Investor Names',
    right_on='Organization/Person Name',
    how='left'
)

distinct_countries = investors['Country'].dropna().unique()[:5]

country_encoded = pd.get_dummies(merged['Country'], prefix='Investor Country', dtype=int)

country_aggregated = country_encoded.groupby(merged['Company ID']).max()

companies = companies.merge(
    country_aggregated,
    left_on='ID',
    right_index=True,
    how='left'
)

for country in distinct_countries:
    column_name = f'Investor Country_{country}'
    if column_name in companies.columns:
        companies[column_name] = companies[column_name].fillna(0)

### Feature 6: Top Investor Participation
Identifies whether a company has investors with a high number of exits.

In [16]:
top_investors = investors[investors['Number of Exits'] > investors['Number of Exits'].mean()]['Organization/Person Name']
def has_top_investor(company_id, funding_df, top_investors):
    company_funding = funding_df[funding_df['Company ID'] == company_id]
    if company_funding.empty:
        return 0
    investors_list = company_funding['Investor Names'].explode()
    return 1 if any(investor in top_investors.values for investor in investors_list) else 0

companies['Top Investor Participation'] = companies['ID'].apply(
    lambda x: has_top_investor(x, funding, top_investors)
)

## New Features: Companies

### Feature 1: 'Months between Founding and Acquisition'

In [17]:
companies['Months between Founding and Acquisition'] = companies.apply(
    lambda row: (row['Announced Date Acquisition'] - row['Founded Date']).days // 30
    if pd.notna(row['Announced Date Acquisition']) else -1,
    axis=1
)

### Feature 2: Category One Hot Encoding

In [18]:
exploded = companies.explode('Industry Groups')
exploded['Industry Groups'] = exploded['Industry Groups'].str.strip().str.lower()
exploded = exploded[exploded['Industry Groups'].notna() & (exploded['Industry Groups'] != '')]

one_hot_encoded = pd.get_dummies(exploded['Industry Groups'], prefix='Industry', dtype=int)
one_hot_aggregated = one_hot_encoded.groupby(exploded.index).max()

companies = pd.concat([companies, one_hot_aggregated], axis=1)

## New Features: LinkedIn-Founder (Aggregation to Company Level)

### Feature 1: Average LinkedIn Followers and Connections

In [19]:
def add_founder_metrics_to_companies(founders, companies):
    founder_metrics = founders.groupby('Company ID').agg(
        average_linkedin_followers_founders=('followers', 'mean'),
        average_linkedin_connections_founders=('connections', 'mean'),
        min_linkedin_followers_founders=('followers', 'min'),
        max_linkedin_followers_founders=('followers', 'max'),
        min_linkedin_connections_founders=('connections', 'min'),
        max_linkedin_connections_founders=('connections', 'max')
    ).reset_index()

    companies = companies.merge(founder_metrics, left_on='ID', right_on='Company ID', how='left')

    for col in ['Company ID', 'Company ID_y']:
        if col in companies.columns:
            companies.drop(columns=[col], inplace=True)

    return companies

companies = add_founder_metrics_to_companies(founders, companies)

## Feature 2: Highest Education

In [20]:
def categorize_highest_education(founders):
    education_mapping = {
        "Doktor/PhD": [
            "phd", "doctor", "doctoral", "dr.", "d.phil", "doctorate", "dsc",
            "dr.phil", "doctor of science", "dr.eng", "ph.d", "ed.d", "sc.d",
            "eng.d", "dr.med", "med.d", "doctor of medicine", "doctor of philosophy",
            "d.ed", "jd", "juris doctor", "doctor juris", "law doctorate", "d.v.m",
            "doctor of veterinary", "md", "m.d.", "doctor of law", "doctor of arts",
            "doctor in", "doctoral studies", "d.lit", "d.m.a", "d.clin.psych",
            "doctor of clinical psychology", "d.jur", "d.theol", "d.b.a",
            "doctor of business administration", "d.eng.sc", "d.arch", "d.d.s",
            "doctor of dental surgery", "d.v.sc", "d.med.sc", "d.p.h",
            "d.sc.tech", "doctor of public health", "d.health.sci", "d.n.p",
            "doctor of nursing practice", "doctor of social work", "doctor of theology",
            "d.comm", "doctor of communication", "d.env.sc", "doctor of environmental science"
        ],
        "Master": [
            "master", "m.sc", "msc", "mba", "m.tech", "ma", "m.eng", "m.ed",
            "ms", "m.phil", "mfa", "m.econ", "mfin", "master of science",
            "master of arts", "master of business", "master's degree",
            "m.des", "m.com", "m.div", "m.theol", "mres", "m.arch", "m.acc",
            "master of engineering", "master of finance", "master of commerce",
            "master of education", "master of philosophy", "master of public health",
            "mph", "m.pp", "m.ir", "mib", "m.int.business", "m.litt", "mchem",
            "diplom", "diploma", "diplom-ing", "diploma in engineering",
            "dipl.-ing", "dipl.-wirtschaftsingenieur", "dipl.-kfm", "dipl.-phys",
            "dipl.-math", "diplomkaufmann", "diplomingenieur", "diplomat",
            "m.comm", "m.a.ed", "m.sc.ed", "m.p.a", "m.h.a", "m.i.s", "m.c.s",
            "master of computer science", "master of information systems",
            "m.sc.tech", "m.plan", "master of planning", "mcm", "master of communication",
            "mhl", "master of human resources", "m.intl.rel", "master of international relations",
            "m.i.t", "m.arch.sc", "master of architecture", "m.journ", "m.fish.sci",
            "master of fisheries science", "m.r.s", "master of rural studies",
            "m.theo", "m.e.e", "master of electrical engineering", "m.med.sc",
            "m.env.sc", "master of environmental science", "m.agri", "master of agriculture",
            "m.p.h.a", "master of public health administration", "m.sc.math"
        ],
        "Bachelor": [
            "bachelor", "b.sc", "bsc", "b.tech", "ba", "b.eng", "b.ed",
            "bs", "b.arch", "bcom", "bba", "bfa", "bpharm", "b.econ",
            "bachelor's degree", "undergraduate", "bcom", "bdes", "bca",
            "bacc", "bachelor of science", "bachelor of arts", "bachelor of technology",
            "bachelor of engineering", "bachelor of education", "bachelor of commerce",
            "bachelor of fine arts", "bachelor of pharmacy", "bachelor of law",
            "bachelor of economics", "llb", "bachelor of computer applications",
            "b.litt", "b.a.e", "b.sc.eng", "b.plan", "b.comm", "b.h.sc",
            "b.i.t", "b.math", "b.stat", "b.mus", "b.of.design", "bcs",
            "bachelor of computing science", "b.eng.tech", "b.a.sc", "b.app.sci",
            "b.e", "b.journ", "b.a.hons", "b.sc.hons", "b.nurs", "b.sc.n",
            "b.soc.sc", "b.soc.work", "bachelor of social work", "b.v.sc",
            "bachelor of veterinary science", "b.med.sc", "b.biochem",
            "bachelor of biochemistry", "b.a.s", "b.env.sc", "bachelor of environmental science",
            "b.med", "bachelor of medicine", "b.optom", "bachelor of optometry",
            "b.psych", "bachelor of psychology", "b.public.health", "b.p.t",
            "bachelor of physical therapy"
        ]
    }

    education_priority = {"Doktor/PhD": 3, "Master": 2, "Bachelor": 1}

    def get_highest_education(degree):
        degree_lower = str(degree).lower()
        for level, keywords in education_mapping.items():
            if any(keyword in degree_lower for keyword in keywords):
                return level
        return None

    founders['degree_1_level'] = founders['degree_1'].apply(get_highest_education)
    founders['degree_2_level'] = founders['degree_2'].apply(get_highest_education)

    founders['degree_1_priority'] = founders['degree_1_level'].map(education_priority).fillna(0)
    founders['degree_2_priority'] = founders['degree_2_level'].map(education_priority).fillna(0)

    founders['highest_education'] = founders.apply(
        lambda row: row['degree_1_level'] if row['degree_1_priority'] >= row['degree_2_priority'] else row['degree_2_level'],
        axis=1
    )

    return founders


def add_highest_education_one_hot_to_companies(founders, companies):
    founders = categorize_highest_education(founders)
    one_hot_encoded = pd.get_dummies(founders['highest_education'], prefix='highest_education')
    founders = pd.concat([founders, one_hot_encoded], axis=1)
    aggregated_one_hot = founders.groupby('Company ID')[one_hot_encoded.columns].sum().reset_index()
    companies = companies.merge(aggregated_one_hot, left_on='ID', right_on='Company ID', how='left')
    companies.fillna(0, inplace=True)
    companies.drop(columns=['Company ID'], inplace=True, errors='ignore')
    return companies

companies = add_highest_education_one_hot_to_companies(founders, companies)

/var/folders/n8/ynbdx9r10lv6l8yh1m_hmbsh0000gn/T/ipykernel_39257/615733937.py:88: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  companies.fillna(0, inplace=True)


In [21]:
def calculate_recognition_rate(founders):
    total_with_degrees = founders[
        founders['degree_1'].notna() | founders['degree_2'].notna()
        ]

    recognized_degrees = total_with_degrees['highest_education'].notna().sum()

    total_with_degrees_count = len(total_with_degrees)
    recognition_rate = recognized_degrees / total_with_degrees_count if total_with_degrees_count > 0 else 0

    return recognition_rate, total_with_degrees_count, recognized_degrees

founders = categorize_highest_education(founders)

recognition_rate, total_with_degrees, recognized_count = calculate_recognition_rate(founders)

print(f"Recognition Rate: {recognition_rate:.2%}")
print(f"Total Entries with Degrees: {total_with_degrees}")
print(f"Recognized Degrees: {recognized_count}")

Recognition Rate: 91.04%
Total Entries with Degrees: 1663
Recognized Degrees: 1514


In [289]:
companies

,ID,Organization Name,Industries,Description,CB Rank (Company),Founded Date,Exit Date,Website,Twitter,Facebook,...,Industry_video,average_linkedin_followers_founders,average_linkedin_connections_founders,min_linkedin_followers_founders,max_linkedin_followers_founders,min_linkedin_connections_founders,max_linkedin_connections_founders,highest_education_Bachelor,highest_education_Doktor/PhD,highest_education_Master
0,000001,2trde,"[Automotive, Software]",2trde develops a software solution designed fo...,"57,123",2017-01-01,0,https://www.2trde.com,0,0,...,0.0,4815.000000,3914.000000,4815.0,4815.0,3914.0,3914.0,0.0,0.0,1.0
1,000002,36ZERO Vision,"[Artificial Intelligence (AI), Computer Vision...",36ZEROVision is an AI-powered visual inspectio...,"51,326",2019-01-01,0,https://36zerovision.com/,0,0,...,0.0,6482.000000,6479.000000,6482.0,6482.0,6479.0,6479.0,0.0,0.0,1.0
2,000003,3Bears Foods,[Food and Beverage],3Bears Foods enable a balanced and delicious b...,"275,817",2015-01-01,0,https://3bears.de/,0,https://www.facebook.com/3bearsporridge/,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,000004,3dTrust,"[3D Printing, Manufacturing, Software]",3dTrust helps companies integrate 3D printing ...,"134,694",2015-01-01,0,http://3dtrust.de,https://twitter.com/3dTrust,0,...,0.0,5664.666667,5337.666667,2089.0,9782.0,1893.0,9238.0,0.0,0.0,2.0
4,000005,abaut,"[Analytics, Artificial Intelligence (AI), Cons...",abaut builds a SaaS that enables businesses al...,"219,525",2017-07-21,0,https://abaut.de,0,0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1508,001506,SAYM,"[Apps, B2B, B2C, Human Resources, Mobile Apps,...",The SAYM platform for swarm mobility defines t...,"166,910",2019-01-01,0,https://www.saym.io/,https://twitter.com/saym_io,https://www.facebook.com/saymcommute/,...,0.0,202.000000,204.000000,202.0,202.0,204.0,204.0,0.0,0.0,1.0
1509,001507,SONAH GmbH,"[Apps, Artificial Intelligence (AI), Computer ...",SONAH developed a flexible embedded vision sen...,"121,752",2016-01-01,0,http://www.sonah.tech,0,0,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1510,001508,Taxy.io,"[FinTech, Legal Tech, Machine Learning, Software]",Taxy.io builds the leading platform for B2B ta...,"220,816",2019-01-01,0,https://www.taxy.io/,https://twitter.com/taxy_io,0,...,0.0,2646.666667,2589.000000,1176.0,5489.0,1116.0,5477.0,0.0,0.0,2.0
1511,001509,TRINKKOST GmbH,"[Agriculture, Consumer Goods, Fitness, Food an...",TRINKKOST is a food supplement manufacturing c...,"907,817",2016-01-01,0,http://www.trinkkost.de,https://www.twitter.com/trinkkost,https://www.facebook.com/trinkkost/,...,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [290]:
founders

,Founder ID,Company ID,Founder Name,Organization Name,first_name,last_name,linkedin_url,logo_url,facebook_url,twitter_url,...,degree_1_university,degree_2,degree_2_university,error,highest_education,degree_1_level,degree_2_level,degree_1_priority,degree_2_priority,is_international
0,000001,000001,Johannes Stoffel,2trde,Johannes,Stoffel,https://www.linkedin.com/in/johannes-stoffel-2...,https://images.crunchbase.com/image/upload/t_c...,NaN,NaN,...,ESCP Business School,"Bachelor of Science - BS, Gerneral Management",EBS Universität für Wirtschaft und Recht,NaN,Master,Master,Master,2.0,2.0,0
2,000002,000002,Florian Ziesche,36ZERO Vision,Florian,Ziesche,None,None,None,None,...,NaN,NaN,NaN,NaN,None,None,None,0.0,0.0,1
3,000003,000002,Zeeshan Karamat,36ZERO Vision,Zeeshan,Karamat,https://www.linkedin.com/in/zkaramat/,https://images.crunchbase.com/image/upload/t_c...,NaN,NaN,...,Technische Universität München,"Masters Computer Science, Artificial Intelligence",Georgia Institute of Technology,NaN,Master,Master,Master,2.0,2.0,1
4,000004,000003,Caroline Steingruber,3Bears Foods,Caroline,Steingruber,None,None,None,None,...,NaN,NaN,NaN,NaN,None,None,None,0.0,0.0,1
5,000005,000003,Tim Nichols,3Bears Foods,Tim,Nichols,None,None,None,None,...,NaN,NaN,NaN,NaN,None,None,None,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3013,003137,001508,Sven Peper,Taxy.io,Sven,Peper,https://www.linkedin.com/in/sven-peper,NaN,NaN,NaN,...,RWTH Aachen University,NaN,NaN,NaN,None,None,None,0.0,0.0,1
3014,003138,001508,Sven Weber,Taxy.io,Sven,Weber,http://www.linkedin.com/in/svenweber,https://images.crunchbase.com/image/upload/t_c...,NaN,NaN,...,Universität Heidelberg,NaN,NaN,NaN,Master,Master,None,2.0,0.0,1
3015,003139,001509,Dr. Gennadi Schechtmann,TRINKKOST GmbH,Dr.,Gennadi Schechtmann,None,None,None,None,...,NaN,NaN,NaN,NaN,None,None,None,0.0,0.0,1
3016,003140,001509,Timon Ortloff,TRINKKOST GmbH,Timon,Ortloff,None,None,None,None,...,NaN,NaN,NaN,NaN,None,None,None,0.0,0.0,1


### Feature 3: International Team

In [22]:
def add_international_team_to_companies(founders, companies):
    founders['is_international'] = founders['country_code'].apply(
        lambda x: 0 if x == 'DEU' or x is None else 1
    )

    international_team = founders.groupby('Company ID')['is_international'].max().reset_index()
    international_team.rename(columns={'is_international': 'international_team'}, inplace=True)

    companies = companies.merge(international_team, left_on='ID', right_on='Company ID', how='left')

    companies['international_team'].fillna(0, inplace=True)
    companies['international_team'] = companies['international_team'].astype(int)

    if 'Company ID' in companies.columns:
        companies.drop(columns=['Company ID'], inplace=True)

    return companies

companies = add_international_team_to_companies(founders, companies)

/var/folders/n8/ynbdx9r10lv6l8yh1m_hmbsh0000gn/T/ipykernel_39257/3700962814.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  companies['international_team'].fillna(0, inplace=True)


### Feature 4: Top University

In [23]:
ranking = pd.read_csv('/Users/janlinzner/Library/Mobile Documents/com~apple~CloudDocs/Documents/McGill/1_Lectures/Decision Analytics (MGSC 662)/2_Coding/Exercises/ADA_Coding/Datasets/Universities/UniversityRanking.csv')
ranking['rank display'] = pd.to_numeric(ranking['rank display'], errors='coerce')
top100 = ranking[ranking['rank display'] <= 100]
top_universities = top100['institution'].dropna().unique().tolist()

def get_top_university_match(uni, top_universities_list, threshold=90):
    if pd.isna(uni) or not uni.strip():
        return None, 0
    match = process.extractOne(uni, top_universities_list, scorer=fuzz.token_set_ratio)
    if match and match[1] >= threshold:
        return match[0], match[1]
    return None, 0

founders['degree_1_top_match'] = None
founders['degree_1_top_score'] = 0
founders['degree_2_top_match'] = None
founders['degree_2_top_score'] = 0

for idx, row in founders.iterrows():
    uni1 = row.get('degree_1_university', '')
    uni2 = row.get('degree_2_university', '')
    match1, score1 = get_top_university_match(uni1, top_universities)
    match2, score2 = get_top_university_match(uni2, top_universities)
    founders.at[idx, 'degree_1_top_match'] = match1
    founders.at[idx, 'degree_1_top_score'] = score1
    founders.at[idx, 'degree_2_top_match'] = match2
    founders.at[idx, 'degree_2_top_score'] = score2

print(founders[['degree_1_university', 'degree_1_top_match', 'degree_1_top_score',
                'degree_2_university', 'degree_2_top_match', 'degree_2_top_score']].head())

def check_top_university(universities, top_universities_list, threshold=90):
    for uni in universities:
        if pd.isna(uni) or not uni.strip():
            continue
        match = process.extractOne(uni, top_universities_list, scorer=fuzz.token_set_ratio)
        if match and match[1] >= threshold:
            return True
    return False

founders['founder_top_uni'] = founders.apply(
    lambda row: check_top_university(
        [row.get('degree_1_university', ''), row.get('degree_2_university', '')],
        top_universities
    ),
    axis=1
).astype(int)

company_top_uni = founders.groupby('Company ID')['founder_top_uni'].mean().reset_index()

companies = companies.merge(company_top_uni, left_on='ID', right_on='Company ID', how='left')
companies['top_university'] = companies['founder_top_uni'].fillna(0)
companies.drop(['Company ID', 'founder_top_uni'], axis=1, inplace=True)

print(companies.head())

/var/folders/n8/ynbdx9r10lv6l8yh1m_hmbsh0000gn/T/ipykernel_39257/577139815.py:28: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '93.87755102040816' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  founders.at[idx, 'degree_1_top_score'] = score1
/var/folders/n8/ynbdx9r10lv6l8yh1m_hmbsh0000gn/T/ipykernel_39257/577139815.py:30: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '95.65217391304348' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  founders.at[idx, 'degree_2_top_score'] = score2


              degree_1_university degree_1_top_match  degree_1_top_score  \
0            ESCP Business School               None                 0.0   
2                             NaN               None                 0.0   
3  Technische Universität München               None                 0.0   
4                             NaN               None                 0.0   
5                             NaN               None                 0.0   

                        degree_2_university               degree_2_top_match  \
0  EBS Universität für Wirtschaft und Recht                             None   
2                                       NaN                             None   
3           Georgia Institute of Technology  Georgia Institute of Technology   
4                                       NaN                             None   
5                                       NaN                             None   

   degree_2_top_score  
0                 0.0  
2             

### Feature 5: Studies Abroad

In [24]:
foreign_universities = ranking[~ranking['location code'].isin(['DE', 'CH', 'AT'])]
foreign_universities = foreign_universities['institution'].dropna().unique().tolist()

def check_foreign_university(universities, foreign_universities_list, threshold=90):
    for uni in universities:
        if pd.isna(uni) or not uni.strip():
            continue
        match = process.extractOne(uni, foreign_universities_list, scorer=fuzz.token_set_ratio)
        if match and match[1] >= threshold:
            return True
    return False

founders['founder_studies_abroad'] = founders.apply(
    lambda row: check_foreign_university(
        [row.get('degree_1_university', ''), row.get('degree_2_university', '')],
        foreign_universities
    ),
    axis=1
).astype(int)

company_abroad = founders.groupby('Company ID')['founder_studies_abroad'].mean().reset_index()

companies = companies.merge(company_abroad, left_on='ID', right_on='Company ID', how='left')
companies['studies_abroad_founder'] = companies['founder_studies_abroad'].fillna(0)
companies.drop(['Company ID', 'founder_studies_abroad'], axis=1, inplace=True)

print(companies[['ID', 'studies_abroad_founder']].head())

       ID  studies_abroad_founder
0  000001                     0.0
1  000002                     0.5
2  000003                     0.0
3  000004                     0.0
4  000005                     0.0


In [25]:
companies

,ID,Organization Name,Industries,Description,CB Rank (Company),Founded Date,Exit Date,Website,Twitter,Facebook,...,min_linkedin_followers_founders,max_linkedin_followers_founders,min_linkedin_connections_founders,max_linkedin_connections_founders,highest_education_Bachelor,highest_education_Doktor/PhD,highest_education_Master,international_team,top_university,studies_abroad_founder
0,000001,2trde,"[Automotive, Software]",2trde develops a software solution designed fo...,"57,123",2017-01-01,0,https://www.2trde.com,0,0,...,4815.0,4815.0,3914.0,3914.0,0.0,0.0,1.0,0,0.000000,0.00
1,000002,36ZERO Vision,"[Artificial Intelligence (AI), Computer Vision...",36ZEROVision is an AI-powered visual inspectio...,"51,326",2019-01-01,0,https://36zerovision.com/,0,0,...,6482.0,6482.0,6479.0,6479.0,0.0,0.0,1.0,1,0.500000,0.50
2,000003,3Bears Foods,[Food and Beverage],3Bears Foods enable a balanced and delicious b...,"275,817",2015-01-01,0,https://3bears.de/,0,https://www.facebook.com/3bearsporridge/,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.000000,0.00
3,000004,3dTrust,"[3D Printing, Manufacturing, Software]",3dTrust helps companies integrate 3D printing ...,"134,694",2015-01-01,0,http://3dtrust.de,https://twitter.com/3dTrust,0,...,2089.0,9782.0,1893.0,9238.0,0.0,0.0,2.0,1,0.333333,0.00
4,000005,abaut,"[Analytics, Artificial Intelligence (AI), Cons...",abaut builds a SaaS that enables businesses al...,"219,525",2017-07-21,0,https://abaut.de,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.000000,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1513,001506,SAYM,"[Apps, B2B, B2C, Human Resources, Mobile Apps,...",The SAYM platform for swarm mobility defines t...,"166,910",2019-01-01,0,https://www.saym.io/,https://twitter.com/saym_io,https://www.facebook.com/saymcommute/,...,202.0,202.0,204.0,204.0,0.0,0.0,1.0,1,0.000000,0.00
1514,001507,SONAH GmbH,"[Apps, Artificial Intelligence (AI), Computer ...",SONAH developed a flexible embedded vision sen...,"121,752",2016-01-01,0,http://www.sonah.tech,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.000000,0.00
1515,001508,Taxy.io,"[FinTech, Legal Tech, Machine Learning, Software]",Taxy.io builds the leading platform for B2B ta...,"220,816",2019-01-01,0,https://www.taxy.io/,https://twitter.com/taxy_io,0,...,1176.0,5489.0,1116.0,5477.0,0.0,0.0,2.0,1,0.250000,0.25
1516,001509,TRINKKOST GmbH,"[Agriculture, Consumer Goods, Fitness, Food an...",TRINKKOST is a food supplement manufacturing c...,"907,817",2016-01-01,0,http://www.trinkkost.de,https://www.twitter.com/trinkkost,https://www.facebook.com/trinkkost/,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.000000,0.00


In [10]:
founders

,Founder ID,Company ID,Founder Name,Organization Name,first_name,last_name,linkedin_url,logo_url,facebook_url,twitter_url,...,degree_1,degree_1_university,degree_2,degree_2_university,error,degree_1_top_match,degree_1_top_score,degree_2_top_match,degree_2_top_score,founder_top_uni
0,1,1,Johannes Stoffel,2trde,Johannes,Stoffel,https://www.linkedin.com/in/johannes-stoffel-2...,https://images.crunchbase.com/image/upload/t_c...,NaN,NaN,...,"Executive Master, Digital Innovation and Entre...",ESCP Business School,"Bachelor of Science - BS, Gerneral Management",EBS Universität für Wirtschaft und Recht,NaN,None,0.0,None,0.0,False
1,2,2,Florian Ziesche,36ZERO Vision,Florian,Ziesche,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,None,0.0,None,0.0,False
2,3,2,Zeeshan Karamat,36ZERO Vision,Zeeshan,Karamat,https://www.linkedin.com/in/zkaramat/,https://images.crunchbase.com/image/upload/t_c...,NaN,NaN,...,"Masters Computer Science, Artificial Intelligence",Technische Universität München,"Masters Computer Science, Artificial Intelligence",Georgia Institute of Technology,NaN,None,0.0,Georgia Institute of Technology,100.0,True
3,4,3,Caroline Steingruber,3Bears Foods,Caroline,Steingruber,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,None,0.0,None,0.0,False
4,5,3,Tim Nichols,3Bears Foods,Tim,Nichols,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,None,0.0,None,0.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2844,3137,1508,Sven Peper,Taxy.io,Sven,Peper,https://www.linkedin.com/in/sven-peper,NaN,NaN,NaN,...,Ingenieurwissenschaften,RWTH Aachen University,NaN,NaN,NaN,None,0.0,None,0.0,False
2845,3138,1508,Sven Weber,Taxy.io,Sven,Weber,http://www.linkedin.com/in/svenweber,https://images.crunchbase.com/image/upload/t_c...,NaN,NaN,...,"Master, Physics",Universität Heidelberg,NaN,NaN,NaN,None,0.0,None,0.0,False
2846,3139,1509,Dr. Gennadi Schechtmann,TRINKKOST GmbH,Dr.,Gennadi Schechtmann,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,None,0.0,None,0.0,False
2847,3140,1509,Timon Ortloff,TRINKKOST GmbH,Timon,Ortloff,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,None,0.0,None,0.0,False
